In [ ]:
%matplotlib inline 
import numpy as np
import autograd as ag
from autograd import numpy as agnp
from autograd import scipy
from scipy.linalg import expm

from sklearn.decomposition import PCA

import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython import display

<h1> Embedded Manifold HMC </h1>

The Geodesic HMC algorithm is a small modification of classic HMC.  The structure of the code is 90% the same as HMC with the modification in that the integration is step is now repeated application of projection and flow steps.

In [ ]:
"""
    HMC but now with corrections to sample on an embedded manifold.

    Inputs:
        initial_params : initial parameters
        n_iters        : number of samples
        epsilon        : leapfrog step size
        L              : number of leapfrog steps
        U              : potential energy w.r.t Hausdorff Measure
        grad_U         : gradient of potential energy w.r.t the Hausdorff Measure
        N              : function for the orthogonal projection basis of the problem
        flow           : geodesic flow update for q and p
        print_iter     : how often to print the sample number
"""
def Embedded_Manifold_HMC(initial_params, n_iters, epsilon, L, U, grad_U, flow, projection, print_iter = 1000):

    chain = [initial_params]
    accepts = 0
    rejects = 0

    for it in range(1, n_iters):
        
        q = chain[it - 1].copy()
        p = np.random.normal(0, 1, size = q.shape)
        #p = p - np.dot(np.dot(N(q), N(q).T),p)
        p = projection(p, q)

        # Compute current potential and kinetic energy
        current_U  = U(q)
        current_K = np.sum(p**2) / 2
       
        for i in range(L):
            p = p - epsilon * grad_U(q) / 2
            #p = p - np.dot(np.dot(N(q),N(q).T),p)
            p = projection(p, q)
            q, p = flow(q, p, epsilon)
            p = p - epsilon * grad_U(q) / 2
            #p = p - np.dot(np.dot(N(q),N(q).T),p)
            p = projection(p,q)

        proposed_U = U(q)
        proposed_K = np.sum(p**2) / 2

        if np.random.uniform(0,1) < np.exp(current_U - proposed_U + current_K - proposed_K):
            chain.append(q)
            accepts += 1
        else:
            chain.append(chain[it-1])
            rejects += 1

        if it % print_iter == 0:
            print(it)

    print(accepts)
    print(rejects)

    return chain

<h1> Geodesic HMC on Stiefel Manifolds </h1>

To implement Geodesic HMC on Stiefel Manifolds we need to define two operations.

1. Projection  
  This defines a projection onto the manifold itself from a position and a velocity.

2. Geodesic Flow  
  This defines the evolution of some time epsilon on the manifold from a starting position.

In [ ]:
# The projection onto the stiefel manifold
# Input: 
#   V: Velocity
#   X: Position
def projection_Stiefel(V,X):
    inner = np.dot(X.T,V) + np.dot(V.T, X)
    return V - (1/2) * np.dot(X,inner)

# The geodesic flow of the particle on the Stiefel Manifold
# Input:
#   X: Position
#   V: Velocity
#   epsilon: The time-step.
def flow_Stiefel(X,V,epsilon):
    A = np.dot(X.T,V)
    S = np.dot(V.T,V)
    mexp = expm(-epsilon * A)
    t1 = expm(epsilon * np.bmat([[A, -S], [np.eye(A.shape[0]), A]]))
    t2 = np.bmat([[mexp, np.zeros(mexp.shape)],[np.zeros(mexp.shape), mexp]])
    F0 = np.bmat([X, V])
    R = np.dot(F0, np.dot(t1,t2))
    X_up, V_up = np.hsplit(R, 2)
    return np.array(X_up), np.array(V_up)

"""
Helper functions to transform angles to the Given's representation
and vice versa
"""
def left_rotate_counter_clockwise(A, angle, i, j):
    AR = A.copy()
    AR[i,:] = np.cos(angle)*A[i,:] - np.sin(angle)*A[j,:]
    AR[j,:] = np.sin(angle)*A[i,:] + np.cos(angle)*A[j,:]
    
    return AR

def right_rotate_counter_clockwise(A, angle, i, j):
    AR = A.copy()
    AR[:,i] = np.cos(angle)*A[:,i] + np.sin(angle)*A[:,j]
    AR[:,j] = -np.sin(angle)*A[:,i] + np.cos(angle)*A[:,j]
    
    return AR

def inverse_givens_transform(angles, n, p):
    G = np.eye(n)
    idx = 0
    for i in range(p):
        for j in range(i+1, n):
            G = right_rotate_counter_clockwise(G, angles[idx], i, j)
            idx = idx + 1
    return G[:,0:p]

def givens_transform(W):
    n, p = W.shape
    angles = [0 for _ in range(int(n*p-p*(p+1)/2))]
    idx = 0
    for i in range(p):
        for j in range(i+1, n):
            angle = np.arctan2(W[j,i],W[i,i])
            W = left_rotate_counter_clockwise(W, -angle, i, j)
            angles[idx] = angle
            idx = idx + 1
    
    return [a/np.pi for a in angles]

<h1> Testing PCA on the Manifold </h1>

Let's look at PCA on some generated data.

In [ ]:
N = 100
n_dim = 2
p_dim = 1

# Generate some 2D data that lies on a line
# Latent representation of the data
z = np.random.normal(0, 1, size = N).reshape((p_dim,N))

# Orthogonal matrix we are trying to infer
W = np.array([[1/np.sqrt(2), 1/np.sqrt(2)]]).T

# Observed data = Wz + \epsilon
data = (np.dot(W, z) + np.random.normal(0, 1, size = (n_dim,N))).T

# The data plotted clearly lies on the line.
plt.scatter(data[:,0], data[:,1])

Now let's apply PCA to it and visualize the found axes.

In [ ]:
m = PCA()
m.fit(data)

data_T = m.transform(data)
c = m.components_

# Visualizing the axes discovered by PCA
plt.scatter(data[:,0], data[:,1])
plt.quiver(0,0,c[0,0],c[0,1], scale = 1/.15, color = 'r')
plt.quiver(0,0,c[1,0],c[1,1], scale = 1/.15, color = 'b')
plt.quiver(0,0,W[0], W[1], scale = 1/.15, color = 'g')
plt.xlim([-2,2])
plt.ylim([-2,2])

Looks pretty self explanatory though it's a bit off for some reason...

<h1> Sampling via embedded manifold HMC </h1>

We will now try to perform the same technique by sampling the PPCA model.

In [ ]:
# The model is y ~ N(Wz + \epsilon)
# This is the PPCA model and the log-likelihood is specified by Tipping and Bishop
# We build the potential energy = -log-likelihood and use autograd to find the derivative
sigma_hat = (1/N) * np.dot(data.T,data)
def U(Q):
    C = agnp.dot(Q, Q.T) + 1
    C_inv = agnp.linalg.inv(C)
    return N/2 * (agnp.log(agnp.linalg.det(C)) + agnp.trace(agnp.dot(C_inv,sigma_hat)))
grad_U = ag.grad(U)

In [ ]:
# Setting up the params for the sampling
initial_params = np.eye(n_dim)[:,:p_dim]
n_iters = 3000
epsilon = 1e-2
L = 100
flow = flow_Stiefel
projection = projection_Stiefel

# Generate samples from the sampler
samples = Embedded_Manifold_HMC(initial_params, n_iters, epsilon, L, U, grad_U, flow, projection) 

In [ ]:
# Visualizing the axes discovered by sampling
plot_samples = False
angles = [givens_transform(d)[0] for d in samples]

if plot_samples:
    plt.scatter(data[:,0], data[:,1])
    plt.quiver(0,0,c[0,0],c[0,1], scale = 1/.15, color = 'g')
    plt.quiver(0,0,c[1,0],c[1,1], scale = 1/.15, color = 'g')

    plt.xlim([-2,2])
    plt.ylim([-2,2])

    for i in range(0,len(samples)):
        d = samples[i]
        angles.append(givens_transform(d)[0])
        plt.scatter(d[0],d[1],color = 'r', alpha = 0.2)
        display.clear_output(wait=True)
        display.display(plt.gcf())
        if i % 20 == 0:
            plt.clf()
            plt.scatter(data[:,0], data[:,1])
            plt.quiver(0,0,c[0,0],c[0,1], scale = 1/.15, color = 'g')
            plt.quiver(0,0,c[1,0],c[1,1], scale = 1/.15, color = 'g')

            plt.xlim([-2,2])
            plt.ylim([-2,2])
        time.sleep(0.0001)

In [ ]:
plt.plot(range(len(angles)), angles)
plt.show()
diff = []
for i in range(1, len(angles)):
    diff.append(angles[i] - angles[i-1])
plt.plot(range(len(angles) - 1), diff)

In [ ]:
plt.hist(angles[1:])
print(np.mean(angles))